In [ ]:
offline_mode="False"

In [ ]:
import logging
import requests
import pandas as pd
import numpy as np
import random
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
wine_df=pd.read_parquet('/opt/airflow/resources/wine_data_raw.parquet')


In [3]:
if offline_mode == "False":
    # map wine type id to wine type
    wine_df["Wine_Category"] = wine_df["Wine_Category"].replace({1: "Red", 2: "White", 3: "Sparkling", 4: "Rosé", 7: "Dessert Wine", 24: "Fortified Wine"})
else:
    wine_df["Wine_Type_ID"]=wine_df["Wine_Type_ID"].replace({1: "Red", 2: "White", 3: "Sparkling", 4: "Rosé", 7: "Dessert Wine", 24: "Fortified Wine"})
    #rename "Wine_type_ID" to "Wine_Category
    wine_df.rename(columns={"Wine_Type_ID":"Wine_Category"}, inplace=True)
    #add a scrape_date column and fill it with 1970-01-01 for each entry
    wine_df["Scrape_Date"]="1970-01-01"
    

# Remove duplicates
wine_df = wine_df.drop_duplicates(subset=['ID'])

In [4]:
problematic_regions_df = pd.DataFrame(columns=['region'])
whitelist_regions_df = pd.DataFrame(columns=['region'])

try:
    # Read the parquet file into a DataFrame
    problematic_regions_df = pd.read_parquet('/opt/airflow/problematic_regions.parquet')

    # Read the whitelist parquet file into a DataFrame
    whitelist_regions_df = pd.read_parquet('/opt/airflow/whitelist_regions.parquet')
except Exception as e:
    print(f"Error occurred while reading the parquet files: {e}")

# Drop rows with regions from the whitelist
if len(whitelist_regions_df) > 0:
    problematic_regions_df = problematic_regions_df[~problematic_regions_df['region'].isin(whitelist_regions_df['region'])]
original_num_rows = len(wine_df)
#print wine_df columns
print(wine_df.columns)
# Drop rows with problematic region names
wine_df = wine_df[~wine_df['Region'].isin(problematic_regions_df['region'])]

# Count the number of final rows dropped
num_rows_dropped = original_num_rows - len(wine_df)

# Print the number of final rows dropped
print(f"Number of final rows dropped: {num_rows_dropped}")


/tmp/ipykernel_994/1589448178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_df['Region'] = wine_df['Region'].str.replace('Grand Cru', '')


In [5]:
#clean NaN values
wine_df['Rating'] = wine_df['Rating'].replace(0, np.nan)
wine_df['Vintage'] = wine_df['Vintage'].replace("N.V.", np.nan)


/tmp/ipykernel_994/793606011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_df['Rating'] = wine_df['Rating'].replace(0, np.nan)
/tmp/ipykernel_994/793606011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_df['Vintage'] = wine_df['Vintage'].replace("N.V.", np.nan)


In [6]:
# drop rows of wine_df['Vintage'] with value '' (empty string)
wine_df = wine_df[wine_df['Vintage'] != '']
#wine_df['Vintage'] = wine_df['Vintage'].fillna(0)
wine_df.dropna(subset=['Vintage'], inplace=True)
wine_df['Vintage'].unique()


array([2016, 2020, 2018, 2014, 2015, 2005, 2009, 2019, 2017, 2004, 1998,
       2011, 1990, 2010, 2012, 2013, 2000, 1989, 1999, 2003, 2006, 2007,
       2008, 2021, 2022, 2001, 1987, 1884, 1982, 1995, 1996, 1986, 1997,
       1985, 2002, 1944, 1924, 1919, 1928, 1893, 1994, 1988, 1993, 1983,
       1970,    0, 1948, 1981, 1984, 1991, 1967, 2023, 1954, 1974, 1959,
       1934, 1980, 1964, 1977, 1961, 1966, 1969, 1992, 1953, 1975, 1976,
       1971, 1979, 1978, 1956, 1936, 1965, 1874, 1951, 1968, 1945, 1960,
       1949, 1962, 1955, 1952, 1950, 1947, 1972, 1973, 1946, 1958, 1963])

In [7]:


# redefine column types for parquet (otherwise it will be object)
wine_df['Vintage'] = wine_df['Vintage'].astype(int)
wine_df['Grape_ID'] = wine_df['Grape_ID'].astype(int)
wine_df['Rating'] = wine_df['Rating'].astype(float)
wine_df['Review_Count'] = wine_df['Review_Count'].astype(int)
wine_df['Price'] = wine_df['Price'].astype(float)
wine_df['Acidity'] = wine_df['Acidity'].astype(float)
wine_df['Fizziness'] = wine_df['Fizziness'].astype(float)
wine_df['Intensity'] = wine_df['Intensity'].astype(float)
wine_df['Sweetness'] = wine_df['Sweetness'].astype(float)
wine_df['Tannin'] = wine_df['Tannin'].astype(float)

wine_df.dtypes
logging.info("Transformation done")


ID                object
Winery            object
Name              object
Vintage            int64
Country           object
Region            object
Wine_Style        object
Wine_Type         object
Wine_Category     object
Grape_Type        object
Grape_ID           int64
Rating           float64
Review_Count       int64
Price            float64
Acidity          float64
Fizziness        float64
Intensity        float64
Sweetness        float64
Tannin           float64
Scrape_Date       object
dtype: object

In [8]:
wine_df.to_parquet('/opt/airflow/resources/wine_data.parquet', engine='fastparquet')

ImportError: Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.